In [1]:
import os

from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from networks.dan import DAN
from collections import OrderedDict
from torchvision import transforms

In [2]:
#labels = ["happy","embarrassed","anger","anxious","hurt","sorrow","neutrality"]
lables = ['happy','surprise','anger','sorrow','neurality']

In [3]:
model = DAN(num_head=4, num_class=5)

In [4]:
model_path = "aihub_epoch39_acc0.8876_bacc0.8885.pth"
checkpoint = torch.load(model_path)
input_size=112

In [5]:
load_weight = checkpoint['model_state_dict']

In [6]:
new_state_dict = OrderedDict()
for n, v in load_weight.items():
    name = n.replace("module.","") 
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model.eval()

DAN(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [7]:
trans = transforms.Compose([
                            transforms.Resize((input_size, input_size)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
                        ])
img = np.random.randint(0, 255, size=(input_size, input_size, 3), dtype=np.uint8)
img = Image.fromarray(img)
img = trans(img)
img = img.view(1,3,input_size,input_size)

In [8]:
opset=11
output = "./dan_230504_e39.onnx"
torch.onnx.export(model, img, output, keep_initializers_as_inputs=False, verbose=False, opset_version=opset)

/data/yoonms/facial_expression/DAN/networks/dan.py:33: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if heads.size(1)>1:


In [ ]:
import onnx

model = onnx.load(output)
graph = model.graph
graph.input[0].type.tensor_type.shape.dim[0].dim_param = 'None'

from onnxsim import simplify
# model, check = simplify(model)
input_shapes = {model.graph.input[0].name : (1, 3, input_size, input_size)}
model, check = simplify(model, input_shapes=input_shapes)
assert check, "Simplified ONNX model could not be validated"

onnx.save(model, output)